In [ ]:
import numpy as np
import math

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [2]:
class Module(object):
    """
    Basically, you can think of a module as of a something (black box) 
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`: 
        
        output = module.forward(input)
    
    The module should be able to perform a backward pass: to differentiate the `forward` function. 
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule. 
    
        gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True
    
    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self, input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.
        
        This includes 
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput
    

    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.
        
        Make sure to both store the data in `output` field and return it. 
        """
        
        # The easiest case:
            
        self.output = input 
        return self.output
       

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input. 
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.
        
        The shape of `gradInput` is always the same as the shape of `input`.
        
        Make sure to both store the gradients in `gradInput` field and return it.
        """
        
        # The easiest case:
        
        self.gradInput = gradOutput 
        return self.gradInput
        
        pass   
    
    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass
    
    def zeroGradParameters(self): 
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass
        
    def getParameters(self):
        """
        Returns a list with its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
        
    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
    
    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [3]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially. 
         
         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`. 
    """
    
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
   
    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)
        
    def remove(self):
        """
        Delete last module from the container.
        """
        self.modules.pop()

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:
        
            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})   
            
            
        Just write a little loop. 
        """
        #self.y_list=[i for i   in range(len(self.modules))] 
        self.output = None
        for ind, module in enumerate(self.modules):
#             if ind ==0:
#                 self.y_list[0]=module.forward(input)
#             else:
#                 self.y_list[ind]=module.forward(self.y_list[ind-1])
            if self.output is None:
                 self.output = module.forward(input)
            else:
                self.output = module.forward(self.output)
                #???
        #raise NotImplementedError
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:
            
            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)   
            gradInput = module[0].backward(input, g_1)   
             
             
        !!!
                
        To each module you need to provide the input, module saw while forward pass, 
        it is used while computing gradients. 
        Make sure that the input for `i-th` layer the output of `module[i]` (just the same input as in forward pass) 
        and NOT `input` to this Sequential module. 
        
        !!!
        
        """
        #self.output=[i for i   in range(len(self.modules))]
        n=len(self.modules)
        self.gradInput = None
        for n in reversed(range(1, len(self.modules))):
        #while n>0:
            #n-=1
#             if n ==len(self.modules)-1:
#                 self.g_list[n]=self.modules[n].backward(self.y_list[n-1], gradOutput)
#                 print('backward from the end')
#             else: 
#                 self.g_list[n]=self.modules[n].backward(self.y_list[n-1], self.g_list[n-1])
            if self.gradInput is None:
            #if n ==len(self.modules)-1:
                self.gradInput=self.modules[n].backward(self.modules[n-1].output, gradOutput)
                print('backward from the end')
            else: 
                self.gradInput=self.modules[n].backward(self.modules[n-1].output, self.gradInput)
        #raise NotImplementedError
        #self.gradInput =  self.modules[0].backward(input, self.g_list[0]) 
        self.gradInput =  self.modules[0].backward(input, self.gradInput) 
        return self.gradInput
      

    def zeroGradParameters(self): 
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]
    
    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self,x):
        return self.modules.__getitem__(x)


# Layers

## 1. Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

You need to **define forward and backward** passes for this layer (updateOutput, updateGradInput, acccGradParameters).

In [4]:
class Linear(Module):
    """
    A module which applies a linear transformation 
    A common name is fully-connected layer, Dense in Keras, Linear in PyTorch. 
    
    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out, learning_rate=0.1):
        super(Linear, self).__init__()
       
        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size=(n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size=n_out)
        
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        self.learning_rate = learning_rate
        
    def updateOutput(self, input):
        """forward pass"""
        self.output=[]
        #print('start linear updateOutput')
        #print(len(input))
        #for ind, i in enumerate(input):
            #print(len(i))
            #per_list=[]
            #для регуляризации добавьте + reg*W к апдейту весов линейного слоя
#             for ind_per, _ in enumerate(self.W):
#                 per=0
#                 #self.output.append(input[ind])
#                 #per.append(input[ind])
#                 #for ind1, _ in enumerate(i):
#                     #print("X=", self.output[ind] )
#                     #print("X[ind1]=", self.output[ind][ind1] )
#                     #print(len(self.W))
#                     #print(len(self.W[0]))
#                     #print("W=", self.W)
#                     #print("W[ind][0]=", self.W[ind1][0])
#                     #self.output[ind][ind1]*=self.W[ind_per][ind1]
#                     per+=input[ind]*self.W[ind_per][ind]
#                 per+=self.b[ind_per]
#                 per_list.append(per)
#             self.output.append(per_list)
        self.output=np.dot(input, self.W.transpose())
        self.output=self.output+self.b
        #print('finish linear updateOutput')
        #print(len(self.output))
        #print(self.output)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        #raise NotImplementedError
        print('back throw linear layer')
        #print('input = ', input)
        #print('input.shape: ', input.shape)
        #self.gradInput = np.dot(self.W.transpose(), gradOutput.transpose())
        #self.gradInput=self.gradInput.transpose()
        self.gradInput = np.dot(gradOutput, self.W)
        #print('gradInput: ', self.gradInput)
        #print('gradInput.shape: ', self.gradInput.shape)
        #print('gradInput.shape: ', self.gradInputt.shape)
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        #raise NotImplementedError
        #print(len(gradOutput))
#         for ind, i in enumerate(self.gradW):
#             self.gradW[ind] -= optimizer_config['learning_rate']*gradOutput
#         self.gradb -= optimizer_config['learning_rate']*gradOutput
        #print(type(input))
        #print('input:', input)
        #print(type(gradOutput))
        #print('gradOutput', gradOutput)
#         for i in range(self.W.shape[0]):
# #             self.gradW[i] *= 0
# #             self.gradb[i] = 0
#             self.zeroGradParameters()
#             for sample, grad in zip(input, gradOutput[i]):
#                 self.gradW[i] += sample * grad
#                 self.gradb[i] += grad
        for sample, grad in zip(input, gradOutput):
            grad1=grad.reshape(-1,1)
            #print('grad reshaped: ', grad1)
            sample1=sample.reshape(1, -1)
            #print('input reshaped: ', sample1)
            self.gradb+=grad
            gradW=grad1 @ sample1
            
#             self.gradW=np.hstack((self.gradW, gradW))
#             self.gradb=np.hstack((self.gradb, grad))
        #print('self.gradW', self.gradW)
        #print('self.gradb', self.gradb)
        
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        return [self.W, self.b]
    
    def getGradParameters(self):
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q
    

## 2. SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

You need to **define forward and backward** passes for this layer (softmax, updateOutput, updateGradInput).

In [5]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()
    
    def softmax(self, output):
        max_ = np.max(output)
        raise NotImplementedError
        softmax_output = None
        return softmax_output
    
    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, np.max(input, axis=1, keepdims=True))
        
        raise NotImplementedError
        self.output = None
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.zeros(input.shape)
        raise NotImplementedError
        return self.gradInput
    
    def __repr__(self):
        return "SoftMax"

## 3. LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

You need to **define forward and backward** passes for this layer (softmax, updateOutput, updateGradInput).

In [ ]:
class LogSoftMax(Module):
    def __init__(self):
         super(LogSoftMax, self).__init__()
    
    def softmax(self, input):
        #raise NotImplementedError
        #print('input ', input)
        input_ = input - np.max(input)
        #print('input_ ', input_)
        sum_ = np.sum(np.exp(input_))
        return np.exp(input_)/sum_
    
    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, np.max(input, axis=1, keepdims=True))
        #raise NotImplementedError
        exp=0
        print('Update LogSoftMax output')
        #print('input:', input)
#         for ind, _ in enumerate(input):
#             for  j in input[ind]:
#                 exp+=math.exp(j)
#             for ind_i, i in enumerate(self.output[ind]):
#                 self.output[ind][ind_i]=(i-math.log(exp))
        for i in range(self.output.shape[0]):
            self.output[i] = self.output[i] - np.log(np.sum(np.exp(self.output[i])))
        return self.output
        #print('logsoftmax=', self.output)
        #return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.zeros(input.shape)
        print('back throw logsoftmax')
        #print('input ', self.gradInput)
        #print('output', gradOutput)
        #for ind, _ in enumerate(input):
            #for ind_j, j in enumerate(input[ind]):
                #(softmax(X) - Y) / batch_size
#         for ind, i in enumerate(input):
#             exp=0
#             for  j in input[ind]:
#                 exp+=math.exp(j)
#             self.gradInput[ind] = (1 - (np.exp(input[ind]) / exp))*gradOutput[ind]
        #self.gradInput = np.zeros(input.shape)
        for i in range(self.gradInput.shape[0]):
            s = self.softmax(input[i])
            #print('s ', type(s) )
            self.gradInput[i] = s / input.shape[0] + gradOutput[i]
        return self.gradInput
        #print('input ', self.gradInput)
        #return self.gradInput
    
    def __repr__(self):
        return "LogSoftMax"
    

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**): 

In [7]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()
    
    def updateOutput(self, input):
        print('update ReLU')
        #print('ReLU input: ', input)
        self.output = np.maximum(input, 0)
        #print('ReLU output: ', self.output)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        print('back throw ReLU')
        #print('gradOutput: ', gradOutput)
        #print('input: ', input)
        self.gradInput = np.multiply(gradOutput , input > 0)
        #print('self.gradInput.shape: ', self.gradInput.shape)
        #print('self.gradInput.shape: ', self.gradInputt)
        return self.gradInput
    
    def __repr__(self):
        return "ReLU"

## 6. Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

You need to **define forward and backward** passes for this layer (updateOutput, updateGradInput).

In [8]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()
            
        self.slope = slope
        
    def updateOutput(self, input):
        #self.output = []
        #raise NotImplementedError
#         for i in input:
#             output_list=[]
#             for j in i:
#                 if j>0:
#                     output_list.append(j)
#                 else:
#                     output_list.append(0.01*j)
#             self.output.append(output_list)
#         return  self.output
        self.output = np.zeros_like(input)
        for i in range(input.shape[0]):
            for j in range(input.shape[1]):
                self.output[i,j] = input[i,j] if input[i,j] > 0 else self.slope*input[i,j]
        return  self.output
    
    
    def updateGradInput(self, input, gradOutput):
        #raise NotImplementedError
        #print('back throw ReLU')
#         for i in input:
#             input_list=[]
#             for j in i:
#                 if j>0:
#                     input_list.append(gradOutput)
#                 else:
#                     input_list.append(0.01*gradOutput)
            #self.gradInput.append(input_list)
        self.gradInput = np.zeros_like(input)
        for i in range(input.shape[0]):
            for j in range(input.shape[1]):
                self.gradInput[i,j] = gradOutput[i,j] if input[i,j] > 0 else self.slope*gradOutput[i,j]
        return self.gradInput    
    
    def __repr__(self):
        return "LeakyReLU"

## 7. ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

You need to **define forward and backward** passes for this layer (updateOutput, updateGradInput).

In [9]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        super(ELU, self).__init__()
        
        self.alpha = alpha
        
    def updateOutput(self, input):
        self.output = np.zeros_like(input)
        for i in range(input.shape[0]):
            for j in range(input.shape[1]):
                self.output[i,j] = input[i,j] if input[i,j] > 0 else self.alpha*(np.exp(input[i,j]) - 1)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.zeros_like(input)
        for i in range(input.shape[0]):
            for j in range(input.shape[1]):
                self.gradInput[i,j] = gradOutput[i,j] if input[i,j] > 0 else self.alpha*np.exp(input[i,j])*gradOutput[i,j]
        return self.gradInput
    
    def __repr__(self):
        return "ELU"

# Criterions

Criterions are used to score the models answers. 

In [10]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
        
    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function 
            associated to the criterion and return the result.
            
            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result. 

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)
    
    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput   

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [11]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()
        
    def updateOutput(self, input, target):   
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output 
 
    def updateGradInput(self, input, target):        
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 9. Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula, 
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

You need to **define forward and backward** passes for this criterion (updateOutput, updateGradInput).

In [12]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def __init__(self):
        a = super(ClassNLLCriterionUnstable, self)
        super(ClassNLLCriterionUnstable, self).__init__()
        
    def updateOutput(self, input, target): 
        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        #raise NotImplementedError
        logP=0
        for ind, i in enumerate(input):
            for ind1, j in enumerate(i):
                #logP=-(target[ind]*log(j)+(1-target[ind])*log(1-j))
                logP+=-target[ind]*log(j)
        self.output = logP/len(target)
        return self.output

    def updateGradInput(self, input, target):
        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        raise NotImplementedError
        self.gradInput = None
        return self.gradInput
    
    def __repr__(self):
        return "ClassNLLCriterionUnstable"
    

## 10. Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

You need to **define forward and backward** passes for this criterion (updateOutput, updateGradInput).

In [13]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()
        
    def updateOutput(self, input, target):
        print('start to calculate criterion')
#         EPS = 1e-15
#         logP=0
#         print('input: ', input)
#         for ind, i in enumerate(input):
#             for ind1, j in enumerate(i):
#                 #logP=-(target[ind]*log(j)+(1-target[ind])*log(1-j))
#                 print('probability: ', j)
#                 print('target[ind][ind1]: ', target[ind][ind1])
#                 #logP+=target[ind][ind1]*math.log(j+EPS)
#                 logP+=target[ind][ind1]*j
#         #self.output = -logP/len(target)
#         self.output = -logP/target.shape[0]
        self.output = target * input
        self.output = -np.sum(self.output) / target.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        #raise NotImplementedError
        #self.gradInput=[]
#         for ind, i in enumerate(target):
#             self.gradInput.append([[1, 1],[1, 1]])
        self.gradInput = -target / input.shape[0]
        #print('NLL gradient ', self.gradInput)
        return self.gradInput
    
    def __repr__(self):
        return "ClassNLLCriterion"
    